# Wearable Health Dashboard
Visualizing health metrics and comparing against benchmarks for a 26-year-old male.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_context("talk")

# Load Data
df = pd.read_csv("Wearable_data/data_processing/apple_health_raw.csv", low_memory=False)

# Filter for relevant metrics
metrics = [
    "HKQuantityTypeIdentifierRestingHeartRate",
    "HKQuantityTypeIdentifierHeartRateVariabilitySDNN",
    "HKQuantityTypeIdentifierVO2Max",
    "HKQuantityTypeIdentifierStepCount"
]

# Filter for Watch data only to avoid double counting (iPhone + Watch)
df = df[df['sourceName'].str.contains('Watch', case=False, na=False)]

df_metrics = df[df["type"].isin(metrics)].copy()
df_metrics["value"] = pd.to_numeric(df_metrics["value"], errors="coerce")
df_metrics["start"] = pd.to_datetime(df_metrics["startDate"])
df_metrics = df_metrics.sort_values("start")

# Separate DataFrames
resting_hr = df_metrics[df_metrics["type"] == "HKQuantityTypeIdentifierRestingHeartRate"]
hrv = df_metrics[df_metrics["type"] == "HKQuantityTypeIdentifierHeartRateVariabilitySDNN"]
vo2 = df_metrics[df_metrics["type"] == "HKQuantityTypeIdentifierVO2Max"]
steps = df_metrics[df_metrics["type"] == "HKQuantityTypeIdentifierStepCount"]

# Daily Aggregates for Steps
steps_daily = steps.set_index("start").resample("D")["value"].sum().reset_index()

In [ ]:
df.type.unique()

In [ ]:
BENCHMARKS = {
    "RestingHR": {"min": 60, "max": 100, "label": "Normal (60-100)"},
    "HRV": {"avg": 60, "label": "roughly 60"},
    "VO2Max": {"avg_min": 41.7, "label": "Fair (>41.7)"},
    "Steps": {"min": 7500, "max": 10000, "label": "Target (7.5k-10k)"}
}

def plot_with_benchmark(ax, data, x_col, y_col, title, ylabel, color, benchmark_key=None, show_good=False):
    if data.empty:
        ax.text(0.5, 0.5, "No Data Available", ha='center', va='center', transform=ax.transAxes)
        ax.set_title(title)
        return

    # 1. Plot the user data first so axes scale automatically
    ax.plot(data[x_col], data[y_col], label="My Data", color=color, alpha=0.8, linewidth=2)
    
    if benchmark_key:
        bm = BENCHMARKS[benchmark_key]
        
        # Handle Range (e.g., Resting HR, Steps)
        if "min" in bm and "max" in bm:
            ax.axhspan(bm["min"], bm["max"], color='green', alpha=0.1, label=bm["label"])
            ax.axhline(bm["min"], color='green', linestyle='--', alpha=0.5)
            ax.axhline(bm["max"], color='green', linestyle='--', alpha=0.5)
        
        # Handle Single Average Line (e.g., HRV)
        if "avg" in bm:
            ax.axhline(bm["avg"], color='orange', linestyle='--', linewidth=2, label=bm["label"])
            
        # Handle VO2 Max specific (Minimum threshold)
        if "avg_min" in bm:
             # Get current chart limits so the shading goes to the top
             current_ylim = ax.get_ylim()
             # Set the shading from the benchmark min up to the chart's current max
             # Use max() to ensure the shading doesn't look cut off if data is low
             top_limit = max(current_ylim[1], bm["avg_min"] + 5) 
             
             ax.axhspan(bm["avg_min"], top_limit, color='green', alpha=0.1, label=bm["label"])
             ax.axhline(bm["avg_min"], color='green', linestyle='--', linewidth=2)
             
    ax.set_title(title, fontsize=16, fontweight='bold')
    ax.set_ylabel(ylabel)
    ax.legend(loc='upper left', frameon=True)
    ax.grid(True, alpha=0.3)

In [ ]:
fig, axes = plt.subplots(4, 1, figsize=(15, 20), sharex=False)

# 1. Resting Heart Rate
plot_with_benchmark(axes[0], resting_hr, "start", "value", 
                   "Resting Heart Rate", "BPM", "#e74c3c", "RestingHR")

# 2. HRV (SDNN)
plot_with_benchmark(axes[1], hrv, "start", "value", 
                   "Heart Rate Variability (SDNN)", "ms", "#8e44ad", "HRV")

# 3. VO2 Max
plot_with_benchmark(axes[2], vo2, "start", "value", 
                   "VO2 Max", "mL/kg/min", "#2980b9", "VO2Max")

# 4. Daily Steps
# Calculate rolling average and mean
steps_daily["rolling_avg"] = steps_daily["value"].rolling(window=30, center=True).mean()
overall_mean = steps_daily["value"].mean()

# Plot raw data
plot_with_benchmark(axes[3], steps_daily, "start", "value", 
                   "Daily Steps", "Steps", "#27ae60", "Steps")
                   
# Add Trend Lines
axes[3].plot(steps_daily["start"], steps_daily["rolling_avg"], 
             color='black', linewidth=2, linestyle='-', label='30-Day Trend')
axes[3].axhline(overall_mean, color='blue', linestyle=':', linewidth=2, label=f'Overall Avg ({overall_mean:.0f})')

# Re-add legend to include new lines
axes[3].legend(loc='upper left', frameon=True)

plt.tight_layout()
plt.show()